In [1]:
# default_exp util

In [2]:
%pip install nbdev | grep -v 'already satisfied'

Note: you may need to restart the kernel to use updated packages.


In [3]:
import lib_py.ipython_magic as ipython_magic

# spiral

In [4]:
%%spi

nominal _obj = $"obj"

inl test b = assert b "test"


<magic:d8f9c0f4186843d8bb9c8dbad627bb30>


<IPython.core.display.Javascript object>

'util.spi'

# node

In [5]:
%%node export

export type PickByType<T, Value> = {
    [P in keyof T as T[P] extends Value | undefined ? P : never]: T[P]
}


'util.ts'

In [6]:
%%node export

import * as fs from "fs"

var start = process.hrtime()

export var elapsed = (start: [number, number]) => process.hrtime(start)[1] / 1000000
export var logStep = (note: string) => {
    console.log(`${process.hrtime(start)[0]} s, ${elapsed(start).toFixed(3)} ms - ${note}`)
    start = process.hrtime()
}
export var sleep = (ms: number) => new Promise(resolve => setTimeout(resolve, ms))
export var waitFileChange = async (path: string) => {
    var start = process.hrtime()
    const stat_mtime = fs.statSync(path).mtimeMs
    while (elapsed(start) < 30000
        && (fs.statSync(path).mtimeMs === stat_mtime
            || fs.readFileSync(path).length === 0)) {
        await sleep(50)
    }
}


'util.ts'

In [7]:
%%node export

export var timeout = async <T>(prom: Promise<T>, ms: number) => {
	let timer
	try {
        return await Promise.race([
            prom,
            new Promise((_r, rej) => timer = setTimeout(rej, ms, new Error('timeout')))
        ])
    } finally {
        return clearTimeout(timer)
    }
}


'util.ts'

# python

In [8]:
#export

import uuid

def new_id():
    return uuid.uuid4().hex

def list_partition(lines, predicate):
    l1, l2 = [], []
    for line in lines:
        (l1, l2)[not predicate(line)].append(line)
    return l1, l2

def write_file(filename, text):
    with open(filename, 'w') as f:
        return f.write(text)

def read_file(filename):
    with open(filename, 'r') as f:
        return f.read()


In [9]:
#export

from time import sleep
from IPython.display import display, Javascript
import subprocess
import os
import uuid
import re

def get_notebook_name():
    if '__vsc_ipynb_file__' in globals():
        ipynb_path = __vsc_ipynb_file__
    else:
        magic = new_id()
        print(f'<magic:{magic}>')

        display(Javascript('IPython.notebook.save_checkpoint();'))

        nb_name = None
        while nb_name is None:
            try:
                sleep(0.1)
                nb_name = subprocess.check_output(f'grep -l {magic} *.ipynb', shell=True).decode().strip()
            except:
                pass

        ipynb_path = os.path.join(os.getcwd(), nb_name)

    ipynb_body = read_file(ipynb_path)
    return re.search(r'\# default_exp (\w+) *', ipynb_body).group(1)

_notebook_name = None


In [10]:
_notebook_name = get_notebook_name()
_notebook_name

'util'

In [11]:
#export

import subprocess

def run_shell(*args, **kwargs):
    try:
        return subprocess.run(*args, shell=True, capture_output=True, **kwargs)
    except Exception as e:
        print(f'util.run_shell() error. args={args} kwargs={kwargs} e={e}')
        raise e
    
def run(*args, **kwargs):
    shell = run_shell(*args, **kwargs)
    err = shell.stderr.decode()
    if err != '':
        print(f'util.run() error. args={args} kwargs={kwargs} err={err}')
        raise Exception(err)
    return shell.stdout.decode().strip('\n')

def run_node(code, timeout=60, *args, **kwargs):
    e = ' --input-type=module -e ' if '\n' in code or '.' not in code else ''
    return run(
        f'''NODE_NO_WARNINGS=1 node --loader ts-node/esm --es-module-specifier-resolution=node {e} \'{code}\'''', 
        *args, 
        timeout=timeout, 
        **kwargs
    )


# py compile

In [12]:
!nbdev_build_lib

Converted 00_util.ipynb.
Converted 01_spiral_api.ipynb.
Converted 02_ipython_magic.ipynb.
Converted 03_algo_network.ipynb.
Converted 04_tictactoe_pyteal.ipynb.
Converted 05_tictactoe_testnet.ipynb.
Converted 06_index.ipynb.
